# Latent Dirichlet Allocation - Advanced Machine Learning

Created a subset of the dataset for news from 2018-2022

 ## Import Libraries and Load Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Gensim is a Python library for topic modelling, document indexing and similarity retrieval with large corpora.
import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess

#The Natural Language Toolkit (NLTK) is a platform used for building Python programs that work with 
# human language data for applying in statistical natural language processing (NLP). 

# It contains text processing libraries for tokenization, parsing, 
# classification, stemming, tagging and semantic reasoning

import nltk
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

from nltk.tokenize import RegexpTokenizer

from nltk.corpus import stopwords
nltk.download('stopwords')
stop=set(stopwords.words('english'))
nltk.download('wordnet')
from nltk.tokenize import word_tokenize

# Convert a collection of text documents to a matrix of token counts.
from sklearn.feature_extraction.text import CountVectorizer

# Convert a collection of raw documents to a matrix of TF-IDF features.
from sklearn.feature_extraction.text import TfidfVectorizer

# This transformer performs linear dimensionality reduction by means of truncated singular value decomposition (SVD). 
# Contrary to PCA, this estimator does not center the data before computing the singular value decomposition. 
# This means it can work with sparse matrices efficiently.
from sklearn.decomposition import TruncatedSVD

# The main goal of this python notebook: Latent Dirichlet Allocation
from sklearn.decomposition import LatentDirichletAllocation

# T-distributed Stochastic Neighbor Embedding.
from sklearn.manifold import TSNE

try:
    import pyLDAvis
    import pyLDAvis.lda_model  
    pyLDAvis.enable_notebook()
except:
    !pip install pyLDAvis
    import pyLDAvis
    import pyLDAvis.lda_model  
    pyLDAvis.enable_notebook()


from collections import  Counter

try:
    from wordcloud import WordCloud
except:
    !pip install wordcloud
    from wordcloud import WordCloud

# Stop warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# A New York Times dataset (Year 2018 - 2022)
df = pd.read_csv("data/nyt_clean.csv")

In [ ]:
df.info()
df.head()

In [ ]:
df['section_name'].nunique()

 ## Preprocess and Explore Data

In [ ]:
""" We will explore the Abstract and Lead Paragraph column one-by-one """

In [ ]:
# Counting how many headline each section have 
New_section_type = df.groupby('section_name').count()['headline'].reset_index()
New_section_type = New_section_type.sort_values(by='headline', ascending=False)
# New_section_type
top_30_section = New_section_type.head(30)
top_30_section_names = top_30_section['section_name'].tolist()
top_30 = df[df['section_name'].isin(top_30_section_names)]

In [ ]:
import plotly.express as px

section_counts = top_30.groupby("section_name").count()["headline"].reset_index()

section_counts['color'] = section_counts['headline'].apply(lambda x: 'red' if x > 7000 else 'blue')

fig = px.scatter(section_counts, x="section_name", y="headline", size="headline", title="Headline based on each section",
                 labels={"headline": "Number of Headlines", "section_name": "Section Name"},
                 size_max=50, color='color')  # Set the color parameter to the new 'color' column

fig.show()

In [ ]:
print("We will explore the World section for the abstract column and Travel section for lead paragraph column-->")

In [ ]:
data1 = df.loc[df['section_name']=='World']
data1.shape

In [ ]:
import textwrap

text = """The exploratory analysis focuses on word frequency, sentence length, average word length analysis, which will help to understand the fundamental characteristics of the text data.

The chart shows nouns that should be no separated as a word single, such as Trump showing a separate word. The next stage will use the Ngram method to solve the problem and mix the words that have the same meaning into one.

Ngrams are defined as contiguous sequences of n words. If the number of words has the same meaning as Donald Trump, it's called Bigram, and if it's more than two words, then it's considered as a trigram.

The chart above shows the top 20 bigrams in the news headline, which is the most frequent words after removing stop words and lemmatization. Comparing the former one, we can see the words more meaningful and not repeated twice, but there still duplicate words in different deadlines. Using bigrams could improve our data retrieval to be more meaningful, but we still need to improve it."""

wrapped_text = textwrap.fill(text, width=80)  # Adjust the width as needed

print(wrapped_text)

In [ ]:
text = """Data cleaning is absolutely crucial for generating a useful topic model: as the saying goes, “garbage in, garbage out.” The steps below are common to most natural language processing methods:

 - Tokenizing: converting a document to its atomic elements.
 - Stopping: removing meaningless words.
 - Lemmatization: Lemmatization is the process of grouping together different inflected forms of the same word."""

paragraphs = text.split('\n')

wrapped_paragraphs = [textwrap.fill(paragraph, width=80) for paragraph in paragraphs]

wrapped_text = '\n'.join(wrapped_paragraphs)

print(wrapped_text)

#### Punctuation Cleaning, Tokenization and Lemmatization

In [ ]:
#function to count the number of words in the article
def word_count(article):
  return len(article.split(" "))

In [ ]:
# Getting word count of each abstract
data1['word_count']=data1['abstract'].apply(word_count) 

In [ ]:
#Total number of words present in the whole abstract corpus 
Total_words1=sum(data1['word_count'])
Total_words1

In [ ]:
#print the sixteenth abstract
data1['abstract'].iloc[15]

In [ ]:
#removing punctuation from the sixteenth abstract
import string
text_without_punctuation = ''.join([char for char in data1['abstract'].iloc[15] if char not in ('"', '“', ',' '”', "'", "—", "’", '.') and char not in string.punctuation])
text_without_punctuation

#added extra characters in the punctuation as “ != ", similarly for other characters

In [ ]:
# tokenizing the sixteenth abstract

# initializing word tokenizer
word_list  = word_tokenize(text_without_punctuation)
word_list

In [ ]:
# lemmatizing the sixteenth abstract

lemma = WordNetLemmatizer()
word_list = [lemma.lemmatize(word) for word in word_list]
word_list

In [ ]:
""" Notice the changes in the words post lemmatizing"""

In [ ]:
#Defining a combined function for all three steps above

def lemmatize_word(text):
    text_without_punctuation = ''.join([char for char in text if char not in ('"', '“', '”', "'", "—", "’", '.') and char not in string.punctuation])
    word_list = word_tokenize(text_without_punctuation)
    lemma = WordNetLemmatizer()
    word_list = [lemma.lemmatize(word) for word in word_list]
    return ' '.join(word_list)

In [ ]:
#Applying lemmatization on the 'abstract' column which will be called as corpus hereafter
data1['abstract']=data1['abstract'].apply(lemmatize_word)

In [ ]:
data1.head()

In [ ]:
#Top 20 most frequent words in the corpus (abstract)
Top_20_freq = pd.Series(' '.join(data1['abstract']).split()).value_counts()[:20]
top20=pd.DataFrame()
top20['Words']= list(Top_20_freq.index)
top20['Counts']=list(Top_20_freq.values)

fig, ax = plt.subplots(figsize=(20,8))
sns.barplot(x = 'Words',y = 'Counts',data = top20,ax=ax).set(title='Top 20 most frequent words in the corpus')

#### Removing short length words ( length less than 3)

In [ ]:
#Removing short length words ( length less than 3)
def short_length(text):
  return (' '.join([wds for wds in text.split() if len(wds)>2]))

data1['abstract']=data1['abstract'].apply(short_length)

In [ ]:
# Total number of words present in the whole corpus
Total_words1 = sum(data1['word_count'])
Total_words1

In [ ]:
# Word count after removal of short length
data1['wc_rem_short_len']=data1['abstract'].apply(word_count) 
display(data1.head())

In [ ]:
"""Notice the word_count and wc_rem_short_len columns in the above dataset"""

#### Removing Stopwords

In [ ]:
#Stopwords

# list of stopwords from nltk
stopwords_nltk = list(stopwords.words('english'))

# # list of stopwords from spacy
# sp = spacy.load('en_core_web_sm')
# stopwords_spacy = list(sp.Defaults.stop_words)

# list of stopwords from gensim
stopwords_gensim = list(gensim.parsing.preprocessing.STOPWORDS)

# unique stopwords from all stopwords
all_stopwords = []
all_stopwords.extend(stopwords_nltk)
# all_stopwords.extend(stopwords_spacy)
all_stopwords.extend(stopwords_gensim)

# all unique stop words
all_stopwords = list(set(all_stopwords))

def stop_words(text):
  text = [wd.lower() for wd in text.split() if wd.lower() not in all_stopwords]
  return " ".join(text)
len(all_stopwords)

In [ ]:
#Removal of Stopwords
data1['abstract']=data1['abstract'].apply(stop_words)
# word count after removing the stopwords
data1['after_stop']=data1['abstract'].apply(word_count)
data1.head() 

In [ ]:
"""Notice the after_stop column and the reduction of the number of stop words. """

In [ ]:
# Cloud Words plot for displaying most frequent words in the corpus (abstract)
freq_words = ','.join(list(data1['abstract'].values))

wordcloud = WordCloud(width=1366, height=768, random_state=21,max_words=100 ,max_font_size=200,background_color='black').generate(freq_words)
plt.figure(figsize=(12, 7))
plt.imshow(wordcloud)
plt.title('Word cloud of most frequent words')
plt.axis('off')
plt.show()

#### Removing of non-contextual words

In [ ]:
#removal of non-contextual words
add_stopwords= ['said','told','called','use','know','came','based','way','added','including','got','week','people','year','new','mr.']
#Function to remove additional stopwords 
def remove_add_stopwords(text):
  text = [wd.lower() for wd in text.split() if wd.lower() not in add_stopwords]
  return " ".join(text)
#Removing additional stopwords according to use case
data1['abstract']=data1['abstract'].apply(remove_add_stopwords)
# word count after removing the additional stopwords
data1['wc_rem_stopword']=data1['abstract'].apply(word_count)
data1.head()   

In [ ]:
#Top 20 most frequent words in the corpus After stopword removal
Top_20_freq = pd.Series(' '.join(data1['abstract']).split()).value_counts()[:20]
top20=pd.DataFrame()
top20['Words']= list(Top_20_freq.index)
top20['Counts']=list(Top_20_freq.values)

fig, ax = plt.subplots(figsize=(20,8))
sns.barplot(x = 'Words',y = 'Counts',data = top20,ax=ax).set(title='Top 20 most frequent words in the corpus After stopword removal')

In [ ]:
"""Notice that the top 20 words have more context"""

#### Visualization of top 20 bi-gram & tri-gram

In [ ]:
def get_top_n_gram(corpus, ngram, n=20):

    ''' 
    This functions takes entire corpus and returns list of tuple in  pair words and number of its occurence.
    '''
    # creates instance of count vectorizer for n-grams
    vec = CountVectorizer(ngram_range=(ngram,ngram)).fit(corpus)

    # returns sparse matrix of index as documents and columns as Bow as features
    bag_of_words = vec.transform(corpus)

    # returns 2D array with count of features in corpus
    sum_words = bag_of_words.sum(axis=0) 

    # returns list of tuples with text,counts pair
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]

    # sort the list of tuples by its number of occurence
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

    # returns list of tuples top n words
    return words_freq[:n]

In [ ]:
# get top 20 bigram words using function
common_words = get_top_n_gram(data1['abstract'],ngram=2)

# creates dataframe from list of tuples 
df1 = pd.DataFrame(common_words, columns = ['Text' , 'count'])

# plot top 20 words
sns.set(font_scale = 1)

fig, ax = plt.subplots(figsize=(20,8))
sns.barplot(y= 'Text',x = 'count',data = df1,ax=ax).set(title='Top 20 bigram words')

In [ ]:
# get top 20 trigram words using function
common_words = get_top_n_gram(data1['abstract'],ngram=3)

# creates dataframe from list of tuples 
df2 = pd.DataFrame(common_words, columns = ['Text' , 'count'])

# plot top 20 words
sns.set(font_scale = 1)

fig, ax = plt.subplots(figsize=(20,8))
sns.barplot(y= 'Text',x = 'count',data = df2,ax=ax).set(title='Top 20 trigram words')

#### Most Frequent words in the corpus show in wordcloud

In [ ]:
#Most Frequent words in the corpus (abstract) show in wordcloud
freq_words = ','.join(list(data1['abstract'].values))

wordcloud = WordCloud(width=1366, height=768, random_state=42,max_words=100 ,max_font_size=200,background_color='black').generate(freq_words)
plt.figure(figsize=(12, 7))
plt.imshow(wordcloud)
plt.title('Most Frequent words in the corpus After stopwords removal',size=20)
plt.axis('off')
plt.show()

In [ ]:
#Data reduction 
after_preprocess=data1['wc_rem_stopword'].sum()
after_preprocess

data_reduce=(after_preprocess/Total_words1)*100
print(f"We have reduced{data_reduce: .02f} percent unnecessary words from our corpus")

In [ ]:
data1.shape

### Lead Paragraph - Preprocessing

In [ ]:
data2 = df.loc[df['section_name']=='Travel']
data2.shape

In [ ]:
# Getting word count of each lead paragraph
data2['word_count']=data2['lead_paragraph'].apply(word_count) 

In [ ]:
#Total number of words present in the whole lead paragraph corpus 
Total_words=sum(data2['word_count'])
Total_words

In [ ]:
# Removing punctuations 

import re
data2['lead_paragraph'] = data2['lead_paragraph'].map(lambda x: re.sub('[,\.!?—’]', '', x)) #removing punctuations
data2['lead_paragraph'] = data2['lead_paragraph'].map(lambda x: x.lower()) # our dataset is already lowercase, this is an additional step here

import gensim
from gensim.utils import simple_preprocess

def preprocess_text(text):
    return gensim.utils.simple_preprocess(str(text), deacc=True)  # deacc=True removes punctuations

# gensim.utils.simple_preprocess lowercases, tokenizes, de-accents (optional). – the output are final tokens = unicode strings, that won’t be processed any further.

data2['lead_paragraph'] = data2['lead_paragraph'].apply(preprocess_text)

In [ ]:
data2['lead_paragraph'].iloc[0]

In [ ]:
#Top 20 most frequent words in the corpus (lead_paragraph)

Top_20_freq = pd.Series(word for sublist in data2['lead_paragraph'] for word in sublist).value_counts()[:20]
top20=pd.DataFrame()
top20['Words']= list(Top_20_freq.index)
top20['Counts']=list(Top_20_freq.values)

fig, ax = plt.subplots(figsize=(20,8))
sns.barplot(x = 'Words',y = 'Counts',data = top20,ax=ax).set(title='Top 20 most frequent words in the corpus')

In [ ]:
# same for lead_paragraph using another method
# Load the regular expression library
import re

# Remove punctuation
data1['lead_paragraph'] = data1['lead_paragraph'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
data1['lead_paragraph'] = data1['lead_paragraph'].map(lambda x: x.lower())

# Print out the first rows of papers
data1['lead_paragraph'].head()


import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = data1.lead_paragraph.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])


# Build the bigram and trigram models
bigram = gensim.models.Phrases( data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
#removing stop_words from lead_paragraph

# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Define functions for stopwords, bigrams, trigrams and lemmatization

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
try:
    !python -m spacy download en_core_web_sm
    import spacy
except Exception as e:
    print(f"An error occurred during spaCy setup: {e}")

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

In [ ]:
#Most Frequent words in the corpus (lead_paragraph) show in wordcloud
freq_words = ' '.join([' '.join(doc) for doc in data_lemmatized])

wordcloud = WordCloud(width=1366, height=768, random_state=42,max_words=100 ,max_font_size=200,background_color='black').generate(freq_words)
plt.figure(figsize=(12, 7))
plt.imshow(wordcloud)
plt.title('Most Frequent words in the corpus After stopwords removal',size=20)
plt.axis('off')
plt.show()

In [ ]:
#removal of non-contextual words from lead_paragraph corpus
add_stopwords= ['say','make','many','take','trip','time','well','day','travel','traveler','know','week','people','year','way','still', 'time']
#Function to remove additional stopwords 
def remove_add_stopwords(words):
    words = [wd.lower() for wd in words if wd.lower() not in add_stopwords]
    return words
# Applying the function to each sublist in data_lemmatized
data_lemmatized = [remove_add_stopwords(doc) for doc in data_lemmatized]

In [ ]:
#Most Frequent words in the corpus (lead_paragraph) show in wordcloud
freq_words = ' '.join([' '.join(doc) for doc in data_lemmatized])

wordcloud = WordCloud(width=1366, height=768, random_state=42,max_words=100 ,max_font_size=200,background_color='black').generate(freq_words)
plt.figure(figsize=(12, 7))
plt.imshow(wordcloud)
plt.title('Most Frequent words in the corpus After stopwords removal',size=20)
plt.axis('off')
plt.show()

## Modelling - Latent Dirichlet Allocation

### Term Frequency Inverse Document Frequency (TF-IDF)

In [ ]:
#Tf-Idf vectoriser

# min_df = 0.03 means that the words (terms) should appear in at least 3% 
# of the documents to be considered in the TF-IDF matrix. words that appear in 
# less than 3% of the documents will be ignored in the TF-IDF matrix.

vectorizer = TfidfVectorizer(max_df=0.95, min_df=0.03) 
document_term_matrix = vectorizer.fit_transform(data1['abstract'])
#Shape of document term matrix
document_term_matrix.shape

In [ ]:
print(document_term_matrix) 

In [ ]:
def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(2, 5, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[-n_top_words:]
        top_features = feature_names[top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f"Topic {topic_idx +1}", fontdict={"fontsize": 30})
        ax.tick_params(axis="both", which="major", labelsize=20)
        for i in "top right left".split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

In [ ]:
from time import time
lda_model = LatentDirichletAllocation(n_components=10,
    max_iter=5,
    learning_method="online",
    learning_offset=50.0,
    random_state=42)

t0 = time()

lda_model.fit(document_term_matrix)

print("done in %0.3fs." % (time() - t0))

tf_feature_names = vectorizer.get_feature_names_out()

plot_top_words(lda_model, tf_feature_names, 20, "Topics in LDA model")

In [ ]:
# Parameters tuning using Grid Search
from sklearn.model_selection import GridSearchCV
grid_params = {'n_components' : list(range(5,10))}

# LDA model
lda_model = GridSearchCV(LatentDirichletAllocation(learning_method ='online', random_state=42),param_grid=grid_params)
lda_model.fit(document_term_matrix)

# Best LDA model
best_lda_model = lda_model.best_estimator_

print("Best LDA model's params" , lda_model.best_params_)
print("Best log likelihood Score for the LDA model",lda_model.best_score_)
print("LDA model Perplexity on train data", best_lda_model.perplexity(document_term_matrix))

In [ ]:
# LDA model fitting for best parameter
lda= LatentDirichletAllocation(n_components=5,learning_method ='online',max_iter=10,random_state=42, n_jobs=-1)
lda.fit(document_term_matrix)

In [ ]:
# LDA model
top_lda=lda.fit_transform(document_term_matrix)

print(top_lda.shape)
print(top_lda)

In [ ]:
# composition of doc 0 for eg
print("Document 0: ")
for i,topic in enumerate(top_lda[0]):
  print("Topic ",i,": ",topic*100,"%")

In [ ]:
print("""The first document is more belonging to the Topic 1.""")

In [ ]:
#lda for lead_paragraph

import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

In [ ]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=42,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [ ]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

NB: The code below works but need long time to run, since we use large dataset. So we note it in case stop kernal by run automatically.

In [ ]:
# import numpy as np
# import tqdm

# grid = {}
# grid['Validation_Set'] = {}

# # Topics range
# min_topics = 2
# max_topics = 11
# step_size = 1
# topics_range = range(min_topics, max_topics, step_size)

# # Alpha parameter
# alpha = list(np.arange(0.01, 1, 0.3))
# alpha.append('symmetric')
# alpha.append('asymmetric')

# # Beta parameter
# beta = list(np.arange(0.01, 1, 0.3))
# beta.append('symmetric')

# # Validation sets
# num_of_docs = len(corpus)
# corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
#                corpus]

# corpus_title = ['75% Corpus', '100% Corpus']

# model_results = {'Validation_Set': [],
#                  'Topics': [],
#                  'Alpha': [],
#                  'Beta': [],
#                  'Coherence': []
#                 }

# # Can take a long time to run
# if 1 == 1:
#     pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
#     # iterate through validation corpuses
#     for i in range(len(corpus_sets)):
#         # iterate through number of topics
#         for k in topics_range:
#             # iterate through alpha values
#             for a in alpha:
#                 # iterare through beta values
#                 for b in beta:
#                     # get the coherence score for the given parameters
#                     cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
#                                                   k=k, a=a, b=b)
#                     # Save the model results
#                     model_results['Validation_Set'].append(corpus_title[i])
#                     model_results['Topics'].append(k)
#                     model_results['Alpha'].append(a)
#                     model_results['Beta'].append(b)
#                     model_results['Coherence'].append(cv)
                    
#                     pbar.update(1)
#     pd.DataFrame(model_results).to_csv('./results/lda_tuning_results.csv', index=False)
#     pbar.close()    

In [ ]:
num_topics = 8

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.9)

In [ ]:
# import pyLDAvis.gensim_models as gensimvis
# import pickle 
# import pyLDAvis
# import os 
# # Visualize the topics
# pyLDAvis.enable_notebook()

# LDAvis_data_filepath = os.path.join('./results/ldavis_tuned_'+str(num_topics))

# # # this is a bit time consuming - make the if statement True
# # # if you want to execute visualization prep yourself
# if 1 == 1:
#     LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
#     with open(LDAvis_data_filepath, 'wb') as f:
#         pickle.dump(LDAvis_prepared, f)

# # load the pre-prepared pyLDAvis data from disk
# with open(LDAvis_data_filepath, 'rb') as f:
#     LDAvis_prepared = pickle.load(f)

# pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_tuned_'+ str(num_topics) +'.html')

# LDAvis_prepared

In [ ]:
import pyLDAvis.lda_model
lda_panel = pyLDAvis.lda_model.prepare(best_lda_model, document_term_matrix,vectorizer,mds='tsne')
lda_panel

In [ ]:
# most important words for each topic
vocab = vectorizer.get_feature_names_out()

In [ ]:
from wordcloud import WordCloud
# Generate a word cloud image for given topic
def word_cloud_lda(index):
  imp_words_topic=""
  comp=lda.components_[index]
  vocab_comp = zip(vocab, comp)
  sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:50]
  for word in sorted_words:
    imp_words_topic=imp_words_topic+" "+word[0]

  wordcloud = WordCloud(width=600, height=400,max_font_size=100).generate(imp_words_topic)
  plt.figure( figsize=(5,5))
  plt.imshow(wordcloud)
  plt.axis("off")
  plt.tight_layout()
  plt.show()
#Word Cloud for each topic
print("""Topic 0""")
word_cloud_lda(0)

In [ ]:
print("""Topic 1""")
word_cloud_lda(1)

In [ ]:
print("""Topic 2""")
word_cloud_lda(2)

In [ ]:
print("""Topic 3""")
word_cloud_lda(3) 

In [ ]:
print("""Topic 4""")
word_cloud_lda(4) 

There are 5 different topic present in the aricles.

LDA and LSA are implemented to find the hidden topics.

LDA performs well & shows 5 different clusters present in the Corpus.